## 인스타그램 크롤링
- 인스타그램 게시물을 크롤링하여 벡터DB에 저장할 데이터를 구축한다.

In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import os
import time
import random

In [2]:
instagramAccountName = "ugly_lovely_official"

In [3]:
def random_sleep(min_sleep = 0.01, max_sleep = 0.2):
    random_sec = random.uniform(min_sleep, max_sleep)
    return random_sec

random_sleep(1,2)

1.9993408835762616

In [4]:
# load .env
load_dotenv()

INSTAGRAM_ID = os.environ.get('INSTAGRAM_ID')
INSTAGRAM_PASSWORD = os.environ.get('INSTAGRAM_PASSWORD')

In [5]:
def click_element_by_xpath(driver, xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        element.click()
    except Exception as e:
        print(f"Error clicking element by XPATH {xpath}: {e}")

def click_element_by_id(driver, element_id):
    try:
        element = driver.find_element(By.ID, element_id)
        element.click()
    except Exception as e:
        print(f"Error clicking element by ID {element_id}: {e}")

def click_element_by_class_name(driver, class_name):
    try:
        element = driver.find_element(By.CLASS_NAME, class_name)
        element.click()
    except Exception as e:
        print(f"Error clicking element by class name {class_name}: {e}")

def click_element_by_LINK_TEXT(driver, link_text):
    try:
        element = driver.find_element(By.LINK_TEXT, link_text)
        element.click()
    except Exception as e:
        print(f"Error clicking element by class name {link_text}: {e}")



In [25]:
# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_experimental_option("excludeSwitches", ['enable-logging'])
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

# 드라이버 생성
# 크롬 드라이버 서비스 생성
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

wait = WebDriverWait(driver, 5)
driver.implicitly_wait(time_to_wait=10)
driver.get("https://instagram.com")
driver.implicitly_wait(time_to_wait=10)


login_id = driver.find_element(By.CSS_SELECTOR, 'input[name="username"]')
login_id.send_keys(INSTAGRAM_ID) # 아이디 입력
random_sleep()
login_pwd = driver.find_element(By.CSS_SELECTOR, 'input[name="password"]')
login_pwd.send_keys(INSTAGRAM_PASSWORD) # 비번 입력
random_sleep()

login_id.send_keys(Keys.ENTER) #enter 키를 쳐주세요
driver.implicitly_wait(10)


In [24]:
def add_address(driver,elements_dict) :
    # BeautifulSoup를 사용하여 페이지 소스 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 모든 게시글 링크 추출
    for a in soup.find_all('a', href=True):
        if a['href'].startswith('/p/'):
            element = "https://instagram.com" + a['href']
            elements_dict[f'{element}'] = element
        if a['href'].startswith('/reel/'):
            element = "https://instagram.com" + a['href']
            elements_dict[f'{element}'] = element


In [26]:
adURL= f"https://instagram.com/{instagramAccountName}"
driver.get(adURL)

# 추가 게시물 확인 및 데이터 로딩
# 페이지 로딩 후 스크롤
SCROLL_PAUSE_TIME = 3
last_height = driver.execute_script("return document.body.scrollHeight")
elements_dict = {}

add_address(driver,elements_dict)

while True:
    # 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 로딩 대기
    time.sleep(SCROLL_PAUSE_TIME)

    add_address(driver,elements_dict)
    print(len(elements_dict))


    # 새로운 높이 계산
    new_height = driver.execute_script("return document.body.scrollHeight")

    # 높이가 변하지 않으면 스크롤 종료
    if new_height == last_height:
        break

    last_height = new_height

24
36
48
60
72
75
75


In [27]:
print(elements_dict)

{'https://instagram.com/p/C86btSNPCr6/': 'https://instagram.com/p/C86btSNPCr6/', 'https://instagram.com/p/C8vhtc4SEpJ/': 'https://instagram.com/p/C8vhtc4SEpJ/', 'https://instagram.com/p/C8eHmypPtwO/': 'https://instagram.com/p/C8eHmypPtwO/', 'https://instagram.com/p/C8WYyJ-OXLV/': 'https://instagram.com/p/C8WYyJ-OXLV/', 'https://instagram.com/p/C8LzDTmOmNm/': 'https://instagram.com/p/C8LzDTmOmNm/', 'https://instagram.com/reel/C8EUa39s_Rr/': 'https://instagram.com/reel/C8EUa39s_Rr/', 'https://instagram.com/p/C8BnLNSPtyS/': 'https://instagram.com/p/C8BnLNSPtyS/', 'https://instagram.com/p/C75yDicu78m/': 'https://instagram.com/p/C75yDicu78m/', 'https://instagram.com/p/C70x0MiOlrz/': 'https://instagram.com/p/C70x0MiOlrz/', 'https://instagram.com/reel/C7xeTQzsKFr/': 'https://instagram.com/reel/C7xeTQzsKFr/', 'https://instagram.com/p/C7nNJu4v8xs/': 'https://instagram.com/p/C7nNJu4v8xs/', 'https://instagram.com/reel/C7i9gpEMfIH/': 'https://instagram.com/reel/C7i9gpEMfIH/', 'https://instagram.co

In [30]:
first_key, first_value = next(iter(elements_dict.items()))
driver.get(elements_dict[first_value])

In [ ]:
# 각 게시글의 텍스트와 이미지를 추출